In [15]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from albums import albums
from time import sleep
import time
from multiprocessing import Pool

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-10-20 18:30:07.766683


In [13]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
albs = albums(disc)
dutils = discogsUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists
Found 100 artist DB files and that is equal to the max mod value


# Collections

In [ ]:
%load_ext autoreload
%autoreload
#col = collections(disc)
#col.parseCollections(force=False)
#col.createCollectionDBs()

# Parse Artists (Parallel)

In [6]:
%load_ext autoreload
%autoreload

from artist import artist
from artists import artists
arts = artists(disc)

def parseArtists(modVal, force=False, doExtra=False):
    #arts.parseArtistModValFiles(modVal, force=force)
    arts.createArtistModValMetadata(modVal=modVal)
    #arts.createArtistAlbumModValMetadata(modVal=modVal)

def parseArtistsParallel(nProcs=3, force=False):
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseArtists, range(100)) 
    while not result.ready():
        if force is True:
            time.sleep(10)
        else:
            time.sleep(1)
    print("")
    return result.get()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")
parseArtistsParallel()
elapsed(start2, cmt2)
elapsed(start, cmt)

Current Time is Sun Oct 13, 2019 20:50:36 for Parsing All Artists
Current Time is Sun Oct 13, 2019 20:50:36 for Parallel Artist Parsing
Saving 5278 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/0-Metadata.p
  --> This file is 315.2kB.
Saving 5344 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/9-Metadata.p
Saving 5502 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/18-Metadata.p
  --> This file is 321.7kB.
  --> This file is 325.4kB.
Saving 5447 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/10-Metadata.p
  --> This file is 326.8kB.
Saving 5573 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/1-Metadata.p
Saving 5385 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/19-Metadata.p
  --> This file is 339.6kB.
  --> This file is 323.0kB.
Saving 5449 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/11-Metadata.p
  --> This file is 328.4kB.

  --> This file is 315.0kB.
Saving 5461 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/75-Metadata.p
  --> This file is 328.7kB.
Saving 5490 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/67-Metadata.p
  --> This file is 330.0kB.
Saving 5546 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/59-Metadata.p
  --> This file is 328.4kB.
Saving 5453 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/76-Metadata.p
  --> This file is 326.6kB.
Saving 5392 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/68-Metadata.p
  --> This file is 320.2kB.
Saving 5439 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/60-Metadata.p
  --> This file is 321.9kB.
Saving 5395 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/77-Metadata.p
  --> This file is 320.6kB.
Saving 5422 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/69-Metadata.p
  --> Th

# Search For New Artists

In [ ]:
arts.searchDiscogForArtist("Diver & Face")

# New Artists From Albums

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums

disc = discogs()
albs = albums(disc)
toget = albs.findMissingArtistsFromAlbums()

In [ ]:
from time import sleep
togetmap  = toget['togetmap']
refstoget = toget['toget']
for ia,item in enumerate(refstoget.most_common()):
    print('\n{0}/{1}'.format(ia,len(refstoget)))
    artistID, cnt = item[0], item[1]
    ref      = togetmap[artistID]
    url      = arts.getArtistURL(ref)
    savename = arts.getArtistSavename(artistID)
    if not isFile(savename):
        try:
            arts.downloadArtistURL(url, savename, debug=True)
        except:
            continue
        sleep(2)
    #print(item)
#len(togetmap)

# Albums

In [ ]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
from discogsBase import discogs

disc = discogs()
albs = albums(disc)
albs.createAlbumModValMetadata(0)

In [ ]:
data = getFile('/Volumes/Music/Discog/albums-db/metadata/0-ArtistMetadata.p')
data

In [16]:
def parseAlbums(modVal):
    albs.parseAlbumModValFiles(modVal=modVal)
    #albs.createAlbumModValMetadata(modVal)
    #albs.createAlbumModValMetadata(modVal=modVal)

def parseAlbumsParallel(modVals=None):
    if modVals is None:
        modVals = ["NAN"] + list(range(100))
        #modVals = range(100)
    nProcs = 3
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseAlbums, modVals)
 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    print("Runing .",end="")
    while not result.ready():
        #print(".",end="")
        time.sleep(10)
    print("")
    
    elapsed(start, cmt)
    return result.get()

In [17]:
%load_ext autoreload
%autoreload
from album import album
from albums import albums
albs = albums(disc)

start,cmt=clock("Parsing All Albums")
start2,cmt2=clock("Parallel Album Parsing")
parseAlbumsParallel()
elapsed(start2, cmt2)
elapsed(start, cmt)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Sun Oct 20, 2019 18:30:35 for Parsing All Albums
Current Time is Sun Oct 20, 2019 18:30:35 for Parallel Album Parsing
Current Time is Sun Oct 20, 2019 18:30:35 for Parsing Album Files For ModVal 8
Current Time is Sun Oct 20, 2019 18:30:35 for Parsing Album Files For ModVal NAN
Current Time is Sun Oct 20, 2019 18:30:35 for Parsing Album Files For ModVal 17
  Looking for artist albums in /Volumes/Music/Discog/albums/8
  Looking for artist albums in /Volumes/Music/Discog/albums/NAN
  Looking for artist albums in /Volumes/Music/Discog/albums/17
Current Time is Sun Oct 20, 2019 18:30:35 for Processing ['NAN', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73

  --> This file is 8.5MB.
Saving 3824 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/17-ArtistMetadata.p
  --> This file is 684.5kB.
Saving 3824 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/17-ArtistAlbums.p
  --> This file is 1.2MB.
Current Time is Sun Oct 20, 2019 18:32:59 for Done with Parsing Album Files For ModVal 17
Process [{0}] took 2.4 minutes.
Current Time is Sun Oct 20, 2019 18:32:59 for Parsing Album Files For ModVal 18
  Looking for artist albums in /Volumes/Music/Discog/albums/18
  Found 5503 artist directories
    Found 3   albums (0    ) in 0/5503	/Volumes/Music/Discog/albums/9/10009
    Found 17  albums (8    ) in 100/5503	/Volumes/Music/Discog/albums/9/1079009
    Found 3   albums (13   ) in 200/5503	/Volumes/Music/Discog/albums/9/1161309
    Found 2   albums (26   ) in 300/5503	/Volumes/Music/Discog/albums/9/1259809
    Found 1   albums (38   ) in 400/5503	/Volumes/Music/Discog/albums/9/1351609
    Found 1   albums (48   ) in 500/5503	/Volum

Current Time is Sun Oct 20, 2019 18:35:27 for Parsing Album Files For ModVal 10
  Looking for artist albums in /Volumes/Music/Discog/albums/10
  Found 2755 artist directories
    Found 1   albums (0    ) in 0/2755	/Volumes/Music/Discog/albums/10/1000810
    Found 3   albums (28   ) in 100/2755	/Volumes/Music/Discog/albums/10/1130910
  --> This file is 10.6MB.
Saving 2811 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/18-ArtistMetadata.p
  --> This file is 662.0kB.
Saving 2811 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/18-ArtistAlbums.p
    Found 1   albums (47   ) in 200/2755	/Volumes/Music/Discog/albums/10/1255810
  --> This file is 1.5MB.
Current Time is Sun Oct 20, 2019 18:35:50 for Done with Parsing Album Files For ModVal 18
Process [{0}] took 2.8 minutes.
Current Time is Sun Oct 20, 2019 18:35:50 for Parsing Album Files For ModVal 19
  Looking for artist albums in /Volumes/Music/Discog/albums/19
    Found 7   albums (63   ) in 300/2755	/Volumes/Music/Di

    Found 0   albums (279  ) in 5534/5535	/Volumes/Music/Discog/albums/19/999919
Saving 279 new album IDs to /Volumes/Music/Discog/albums-db/19-DB.p
  Found 3728 artist directories
    Found 2   albums (0    ) in 0/3728	/Volumes/Music/Discog/albums/11/1000811
    Found 1   albums (11   ) in 100/3728	/Volumes/Music/Discog/albums/11/1104411
    Found 28  albums (20   ) in 200/3728	/Volumes/Music/Discog/albums/11/1200911
    Found 4   albums (33   ) in 300/3728	/Volumes/Music/Discog/albums/11/1306711
    Found 5   albums (36   ) in 400/3728	/Volumes/Music/Discog/albums/11/140811
    Found 1   albums (42   ) in 500/3728	/Volumes/Music/Discog/albums/11/1540211
    Found 1   albums (49   ) in 600/3728	/Volumes/Music/Discog/albums/11/1641011
    Found 2   albums (65   ) in 700/3728	/Volumes/Music/Discog/albums/11/1747011
    Found 4   albums (71   ) in 800/3728	/Volumes/Music/Discog/albums/11/18911
    Found 0   albums (79   ) in 900/3728	/Volumes/Music/Discog/albums/11/1989011
    Found 1   

    Found 0   albums (77   ) in 1600/4955	/Volumes/Music/Discog/albums/12/2237812
    Found 5   albums (78   ) in 1700/4955	/Volumes/Music/Discog/albums/12/234212
    Found 0   albums (85   ) in 1800/4955	/Volumes/Music/Discog/albums/12/2431512
    Found 0   albums (90   ) in 1900/4955	/Volumes/Music/Discog/albums/12/252012
    Found 4   albums (92   ) in 2000/4955	/Volumes/Music/Discog/albums/12/2607912
    Found 1   albums (96   ) in 2100/4955	/Volumes/Music/Discog/albums/12/2689812
    Found 9   albums (100  ) in 2200/4955	/Volumes/Music/Discog/albums/12/2784112
    Found 6   albums (115  ) in 2300/4955	/Volumes/Music/Discog/albums/12/29112
    Found 0   albums (127  ) in 2400/4955	/Volumes/Music/Discog/albums/12/301212
  --> This file is 8.1MB.
Saving 2766 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/20-ArtistMetadata.p
    Found 1   albums (131  ) in 2500/4955	/Volumes/Music/Discog/albums/12/3102912
  --> This file is 631.8kB.
Saving 2766 new artist IDs to /Volumes/M

...

  --> This file is 757.0kB.
Saving 3840 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/92-ArtistAlbums.p
  --> This file is 1.2MB.
Current Time is Sun Oct 20, 2019 19:55:33 for Done with Parsing Album Files For ModVal 92
Process [{0}] took 2.1 minutes.
Current Time is Sun Oct 20, 2019 19:55:34 for Parsing Album Files For ModVal 93
  Looking for artist albums in /Volumes/Music/Discog/albums/93
  --> This file is 7.9MB.
Saving 2789 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/87-ArtistMetadata.p
  --> This file is 638.9kB.
Saving 2789 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/87-ArtistAlbums.p
  --> This file is 1.1MB.
Current Time is Sun Oct 20, 2019 19:55:47 for Done with Parsing Album Files For ModVal 87
Process [{0}] took 2.1 minutes.
Current Time is Sun Oct 20, 2019 19:55:47 for Parsing Album Files For ModVal 88
  Looking for artist albums in /Volumes/Music/Discog/albums/88
    Found 0   albums (2881 ) in 400/5566	/Volumes/Music/Discog/albu

    Found 2   albums (192  ) in 5000/5588	/Volumes/Music/Discog/albums/88/6567488
    Found 0   albums (193  ) in 5100/5588	/Volumes/Music/Discog/albums/88/6912688
    Found 5   albums (197  ) in 5200/5588	/Volumes/Music/Discog/albums/88/729388
    Found 1   albums (296  ) in 3700/3808	/Volumes/Music/Discog/albums/93/895493
    Found 3   albums (219  ) in 5300/5588	/Volumes/Music/Discog/albums/88/801088
    Found 2   albums (306  ) in 3800/3808	/Volumes/Music/Discog/albums/93/993
    Found 3   albums (307  ) in 3807/3808	/Volumes/Music/Discog/albums/93/99793
Saving 307 new album IDs to /Volumes/Music/Discog/albums-db/93-DB.p
    Found 6   albums (239  ) in 5400/5588	/Volumes/Music/Discog/albums/88/861488
    Found 0   albums (256  ) in 5500/5588	/Volumes/Music/Discog/albums/88/92488
    Found 2   albums (270  ) in 5587/5588	/Volumes/Music/Discog/albums/88/999988
Saving 270 new album IDs to /Volumes/Music/Discog/albums-db/88-DB.p
    Found 4   albums (3926 ) in 600/5566	/Volumes/Music/D

    Found 1   albums (263  ) in 3200/3526	/Volumes/Music/Discog/albums/94/713594
    Found 0   albums (274  ) in 3300/3526	/Volumes/Music/Discog/albums/94/788094
    Found 1   albums (291  ) in 3400/3526	/Volumes/Music/Discog/albums/94/88394
    Found 7   albums (299  ) in 3500/3526	/Volumes/Music/Discog/albums/94/976494
    Found 1   albums (300  ) in 3525/3526	/Volumes/Music/Discog/albums/94/9994
Saving 300 new album IDs to /Volumes/Music/Discog/albums-db/94-DB.p
  --> This file is 7.6MB.
Saving 3742 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/98-ArtistMetadata.p
  --> This file is 736.2kB.
Saving 3742 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/98-ArtistAlbums.p
    Found 2   albums (5588 ) in 1000/5566	/Volumes/Music/Discog/albums/79/1762679
  --> This file is 1.1MB.
Current Time is Sun Oct 20, 2019 19:59:55 for Done with Parsing Album Files For ModVal 98
Process [{0}] took 1.8 minutes.
Current Time is Sun Oct 20, 2019 19:59:55 for Parsing Album Files 

Saving 3996 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/99-ArtistAlbums.p
    Found 2   albums (274  ) in 3400/3791	/Volumes/Music/Discog/albums/95/686795
    Found 2   albums (285  ) in 3500/3791	/Volumes/Music/Discog/albums/95/728995
  --> This file is 1.1MB.
Current Time is Sun Oct 20, 2019 20:01:38 for Done with Parsing Album Files For ModVal 99
Process [{0}] took 1.7 minutes.
    Found 3   albums (301  ) in 3600/3791	/Volumes/Music/Discog/albums/95/825595
    Found 1   albums (320  ) in 3700/3791	/Volumes/Music/Discog/albums/95/912395
    Found 2   albums (325  ) in 3790/3791	/Volumes/Music/Discog/albums/95/999295
Saving 325 new album IDs to /Volumes/Music/Discog/albums-db/95-DB.p
    Found 7   albums (7111 ) in 1400/5566	/Volumes/Music/Discog/albums/79/2069179
  --> This file is 9.1MB.
Saving 3791 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/95-ArtistMetadata.p
  --> This file is 745.5kB.
Saving 3791 new artist IDs to /Volumes/Music/Discog/albums-db/m

    Found 0   albums (242  ) in 3200/3570	/Volumes/Music/Discog/albums/97/691597
    Found 3   albums (246  ) in 3300/3570	/Volumes/Music/Discog/albums/97/744197
    Found 61  albums (271  ) in 3400/3570	/Volumes/Music/Discog/albums/97/84097
    Found 3   albums (289  ) in 3500/3570	/Volumes/Music/Discog/albums/97/922597
    Found 1   albums (300  ) in 3569/3570	/Volumes/Music/Discog/albums/97/999297
Saving 300 new album IDs to /Volumes/Music/Discog/albums-db/97-DB.p
    Found 0   albums (9751 ) in 2200/5566	/Volumes/Music/Discog/albums/79/2739579
  --> This file is 8.3MB.
Saving 3570 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/97-ArtistMetadata.p
  --> This file is 736.4kB.
Saving 3570 new artist IDs to /Volumes/Music/Discog/albums-db/metadata/97-ArtistAlbums.p
  --> This file is 1.3MB.
Current Time is Sun Oct 20, 2019 20:05:04 for Done with Parsing Album Files For ModVal 97
Process [{0}] took 1.4 minutes.
    Found 7   albums (10191) in 2300/5566	/Volumes/Music/Discog/

In [ ]:
db = getFile('/Volumes/Music/Discog/albums-db/metadata/89-ArtistMetadata.p')
db

# New Albums

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
albs = albums(disc)
albs.downloadAlbumsFromArtists(maxAlbums=3, mediaTypes=None, rand=True)

In [ ]:
for k,v in db.items():
    print(k,v)
    break

In [ ]:
f1="/Volumes/Music/Discog/albums/6/102506/482739.p"
#f2="/Volumes/Music/Discog/albums/6/102506/7303693.p"
#albData1 = alb.getData(f1)
f2="/Volumes/Music/Discog/albums/11/567511/3386791.p"
albData2 = alb.getData(f2)

In [ ]:
albData1.album

In [ ]:
for k,v in albData2.get().items():
    try:
        print(k,v.get())
    except:
        print(k,v)

In [ ]:
for k,v in albData1.get().items():
    try:
        print(k,v.get())
    except:
        print(k,v)

In [ ]:
from album import album
from fileUtils import getsize
alb = album()
for ifile in files:
    print(getsize(ifile))
    #alb.getData(ifile)

In [ ]:
for modVal in range(58,59):
    arts.parseArtistModValFiles(modVal=modVal, force=False)

# New Artists From Albums

In [ ]:
toget = getFile('/Users/tgadfort/Documents/code/music/toget.p')
toget2 = getFile('/Users/tgadfort/Documents/code/music/toget2.p')
toget.update(toget2)

for ia,(artistID,artistRef) in enumerate(toget.items()):
    url = arts.getArtistURL(artistRef)
    savename = arts.getArtistSavename(artistID)
    try:
        arts.downloadArtistURL(url, savename, debug=True, force=True)
    except:
        continue
    if ia % 10 == 0:
        print("\nProcessed {0}/{1} Files\n".format(ia,len(toget)))

dbArt = disc.getArtistIDToNameData()
dbCol = disc.getCollectionIDToRefData()
artistDBDir = disc.getArtistsDBDir()

In [ ]:
from time import sleep
refs = [dbCol.get('716158')]
for ref in refs:
    if ref is None:
        continue
    print("REF --> {0}".format(ref))
    href = ref.replace("https://www.discogs.com/artist/", "")
    artistID = dutils.getArtistID(ref)
    modValue = dutils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())

    if True:
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modValue))     
        print("Loading {0}".format(dbname))
        dbdata  = getFile(dbname)

        try:
            del dbdata[artistID]
            print("Deleted {0}".format(artistID))
        except:
            print("Not there...")

        print("Saving {0}".format(dbname))
        saveFile(idata=dbdata, ifile=dbname)

    url = arts.getArtistURL(ref)
    savename = arts.getArtistSavename(artistID)
    arts.downloadArtistURL(url=url, savename=savename, parse=True, force=True, debug=True)
    sleep(1)

In [ ]:
import time
from multiprocessing import Pool
 
def f(a_list):
    out = 0
    for n in a_list:
        out += n*n
        time.sleep(0.1)
 
    return out
 
def f_mp(a_list):
    chunks = [a_list[i::5] for i in range(5)]
 
    pool = Pool(processes=5)
 
    result = pool.map(f, chunks)
 
    return sum(result)

In [ ]:
import time
from multiprocessing import Pool
 
def f(a_list):
    out = 0
    for n in a_list:
        out += n*n
        time.sleep(0.1)
 
    return out
 
def f_amp(a_list):
    chunks = [a_list[i::5] for i in range(5)]
 
    pool = Pool(processes=5)
 
    result = pool.map_async(f, chunks)
 
    while not result.ready():
        print("Running...")
        time.sleep(0.5)
 
    return sum(result.get())

In [ ]:
L = [{0: True}, {1: True}, {2: True}, {3: True}, {4: True}]

In [ ]:
{k: v for d in L for k, v in d.items()}

In [ ]:
import time
from multiprocessing import Pool
 
artistDir = disc.getArtistsDir()
maxModVal = disc.getMaxModVal()
artistInfo = artist()
artistDBDir = disc.getArtistsDBDir()   

In [ ]:
len(files)

In [ ]:
from fileUtils import getBaseFilename
import time
from multiprocessing import Pool

def f(finput):
    a_list = finput["Data"]
    dbdata = finput["DB"]
    out = []
    for ifile in a_list:
        discID = getBaseFilename(ifile)
        if dbdata.get(discID) is None:
            out.append(ifile)
            
    return out

def fDB(modVal):
    saveIt = arts.parseArtistModValFiles(modVal, force=True)
    
    if False:
        from searchUtils import findExt
        files = findExt(setDir(artistDir, str(modVal)), ext="*.p")
        dbname = setFile(artistDBDir, "{0}-DB.p".format(modVal))
        dbdata = getFile(dbname, version=3)

        out = []
        saveIt = 0
        for ifile in files:
            discID = getBaseFilename(ifile)
            if dbdata.get(discID) is None:
                info   = artistInfo.getData(ifile)
                dbdata[discID] = info
                saveIt += 1

        if saveIt > 0:
            savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))
            print("Saving {0} new artist IDs to {1}".format(saveIt, savename))
            saveJoblib(data=dbdata, filename=savename, compress=True)
        
    return saveIt

 
def f_ampFiles(a_list, dbdata):
    nProcs = 1
    chunks = [a_list[i::nProcs] for i in range(nProcs)]

    for c in chunks:
        print(len(c))
    
    pool = Pool(processes=nProcs)
    
    
 
    finputs = [{"Data": chunk, "DB": dbdata} for chunk in chunks]
    result = pool.map_async(f, finputs)
 
    start, cmt = clock("Processing {0} files using {1} processes".format(len(files), nProcs))
    print("Runing .",end="")
    while not result.ready():
        print(".",end="")
        time.sleep(1)
    print("")
    
    elapsed(start, cmt)
    return result.get()



 
def f_amp(modVals):
    nProcs = 3
    pool = Pool(processes=nProcs)
    result = pool.map_async(fDB, range(3,modVals))
 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    print("Runing .",end="")
    while not result.ready():
        print(".",end="")
        time.sleep(1)
    print("")
    
    elapsed(start, cmt)
    return result.get()

In [ ]:
from timeUtils import clock, elapsed
retval = f_amp(100)
#flat_list = [item for sublist in retval for item in sublist]
#print(len(flat_list))

In [ ]:
dbname = setFile(artistDBDir, "{0}-DB.p".format(67))            
dbdata = getFile(dbname, version=3)

In [ ]:
modVals = 1
nProcs = 1
start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
print("Runing .",end="")
retval = fDB(modVals)
print("")
elapsed(start, cmt)


In [ ]:
retval

In [ ]:
vals="""1,708,466 US
1,708,466 UK
1,154,284 Germany
637,261 France
493,079 Japan
490,698 Italy
434,681 Canada
433,543 Europe
416,965 Netherlands
339,738 Spain
320,714 Unknown
259,587 Russia
259,579 Australia
189,431 Sweden
164,278 Belgium
149,548 Brazil
131,106 Greece
124,488 Poland
121,249 Jamaica
105,887 Finland
86,538 Switzerland
85,145 USSR
80,203 Mexico
79,972 Denmark
77,131 Portugal
75,979 Norway
75,533 UK & Europe
67,292 Austria
63,672 Argentina"""

In [ ]:
countries = [" ".join(x.split()[1:]) for x in vals.split("\n")]
saveFile(idata=countries, ifile="countries.p")

In [ ]:
countries